In [ ]:
#| default_exp methods.causal

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.data import TabularDataModule, TabularDataModuleConfigs
from relax.methods.base import BaseCFModule
from relax.utils import validate_configs, binary_cross_entropy, grad_update

In [ ]:
#| export
class CausalCFConfig(BaseParser):
    optimization_approach: str = 'brute_force'
    n_samples: int = 10

In [ ]:
#| export
def get_intervenable_nodes(dm: TabularDataModule) -> list:
    """Get the list of nodes that can be intervened."""

    intervenable_nodes = {
        "continuous": np.setdiff1d(
            dm._configs.continous_cols, dm._configs.immutables
        ),
        "categorical": np.setdiff1d(
            dm._configs.discret_cols, dm._configs.immutables
        ),
    }
    return intervenable_nodes

In [ ]:
#| export
def action_set_cost(
    X: jnp.ndarray,
    action_set: dict,
    norm: int = 2
) -> float:
    """Calculate the cost of an action set."""

    deltas = action_set - X
    return jnp.linalg.norm(deltas, ord=norm)

We assume [0,1] scalar.

In [ ]:
list(dict.fromkeys(['a', 'b', 'c', None]))

['a', 'b', 'c', None]

In [ ]:
#| export
class CausalCF(BaseCFModule):
    name = "CausalCF"

    def __init__(
        self,
        configs: dict | CausalCFConfig = None
    ):
        if configs is None: 
            configs = CausalCFConfig()
        self.configs = validate_configs(configs, CausalCFConfig)

    def compute_optimal_action_set(
        self,
        X: jnp.ndarray, # Input to be explained
        pred_fn: Callable = None # Predictive function
    ) -> jnp.ndarray:
        """Compute optimal action set."""

        intervenable_nodes = get_intervenable_nodes(self._data_module)
        
        if self.configs.optimization_approach == 'brute_force':
            return self._brute_force(intervenable_nodes, X, pred_fn)
        elif self.configs.optimization_approach == 'gradient_descent':
            raise NotImplementedError
        else:
            raise ValueError(f"optimization approach {self.configs.optimization_approach} "
                             "is not recognized")
    
    def _brute_force(
        self,
        intervenable_nodes: list,
        X: jnp.ndarray, # Input to be explained
        pred_fn: Callable = None # Predictive function
    ):
        min_val, max_val, mean_val = \
            np.min(X, axis=-1), np.max(X, axis=-1), np.mean(X, axis=-1)
        min_cost, min_action_set = np.inf, {}

        valid_action_sets = get_discretized_action_sets(
            intervenable_nodes, min_val, max_val, mean_val
        )

        for action_set in valid_action_sets:
            if constraint_handle(
                self._scm, X, action_set, sampling_handle, pred_fn,
            ):
                cost = action_set_cost(X, action_set)
                if cost < min_cost:
                    min_cost = cost
                    min_action_set = action_set
        return min_action_set, min_cost
    
    def _validate_input_range(
        self,
        X: jnp.ndarray, # Input to be explained
    ):
        if not np.all((1 > X) & (X > 0)):
            raise ValueError("Input must be in [0, 1] range.")
        
    def _hook_scm(
        self, dm: TabularDataModule
    ):
        if not hasattr(dm, "_scm"):
            raise ValueError("DataModule does not have a `scm` attribute.")
        self._scm = dm._scm

    def before_generate_cfs(
        self,
        dm: TabularDataModule, # data module
        X: jnp.ndarray, # Input to be explained
    ):
        """Hook to be called before `generate_cfs`."""
        self._hook_scm(dm)
        self._validate_input_range(X)
    
    def generate_cfs(
        self,
        X: jnp.ndarray, # Input to be explained
        pred_fn: Callable = None # Predictive function 
    ) -> jnp.ndarray:
        """Generate counterfactuals."""
        self.before_generate_cfs(self.data_module, X)

        cfs = []
        for i in range(self.configs.n_samples):
            action_set, _ = self.compute_optimal_action_set(X)
            cf = X + action_set # TODO: implement array_plus_dict
            cfs.append(cf)
        return jnp.concatenate(cfs, axis=0)
